In [ ]:
import numpy as np
import scipy.stats as stats
from hansolo import *
import random

np.random.seed(12)

Nb = 100
expert = "PWA"  # "EWA" or "PWA" or "Multilin"
learning="blocs" # "blocs": objects presented by random blocs, or "random": each object is randomly chosen with replacement


M = 2502
dt = 0.002
N = 1000
T = N * dt

freq = 100  # firing rate input neurons +
freq2 = 150

alpha = np.array([100 * dt, 0])  # spontaneous rates of output neurons

p = freq * dt  # spiking proba input neurons
p2 = freq2 * dt

K_input = 12  # nb experts
n_input = 6  # nb input neurons
K_output = 2  # nb output neurons

sup = freq2 * dt * ((9 / 8) + 9)
eta_output = np.sqrt(8 * np.log(K_input) / M) / sup  # para EWA

n_feat = 6
N_a=n_feat


obj = [
    [1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1],
    [0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1],
]
# blue circle blue square blue triangle red circle red square red triangle gray circle gray square gray triangle



answers = np.zeros((Nb, M, N_a))  # 0 if wrong, 1 if correct

W_output = np.zeros((K_output, K_input, M, Nb, N_a)) - 1


# para_PWA=2*np.log(K_input)
para_PWA = 2

for n_a in range(N_a):
    n_input_new = n_input - n_a
    K_input_new = 2*n_input_new
    F_output = np.zeros((K_output, M, Nb))  # firing rate output neurons


    for nb in range(Nb):
        feat = np.sort(random.sample(range(n_feat), n_feat - n_a)) #features that we keep
        
        feat2 = np.concatenate((feat,feat+n_feat))
        W_output[:, feat2, 0, nb, n_a] = 1 / K_input_new

        if learning == 'blocs':
            L=list_obj_random(obj,M)
        else:
            L=list_objects_random(obj,M)

        W_output_not_renorm = np.zeros((K_output, K_input_new, M))
        W_output_not_renorm[:, :, 0] += 1

        F_input = np.zeros((n_input_new, M))

        cred_cum_output = np.zeros(K_output)
        cred_cum_input = np.zeros((K_output, K_input_new))

        for m in range(M):
            obj_cur = L[m]
            activity_cur = obj_cur[feat]

            # simulation of the input neurons
            input_neurons = np.zeros((n_input_new, N))
            cur_act = np.where(activity_cur == 1)[0]
            input_neurons[cur_act, :] = Poisson(freq, T, (cur_act.size, N))

            # simulation of the output neurons
            output_neurons = np.zeros((K_output, N))
            for j in range(K_output):
                
                probas = alpha[j] + np.sum(
                    (
                        W_output[j, feat, m, nb, n_a][:, np.newaxis]
                        - W_output[j, feat + n_feat, m, nb, n_a][:, np.newaxis]
                    )
                    * input_neurons,
                    axis=0,
                )
                clipped_probas = np.clip(probas, 0, 1)
                #print(clipped_probas)
                output_neurons[j, :] = stats.bernoulli.rvs(clipped_probas)

            # firing rates
            F_input[:, m] = np.sum(input_neurons, axis=1) / T
            F_output[:, m, nb] = np.sum(output_neurons, axis=1) / T

            if (in_A(obj_cur) and F_output[0, m, nb] > F_output[1, m, nb]) or (
                in_B(obj_cur) and F_output[1, m, nb] > F_output[0, m, nb]
            ):
                answers[nb, m, n_a] = 1

            cred = cred_output_HAN(K_output, K_input_new, F_input[:, m], dt, obj_cur)
            cred_cum_output += np.sum(W_output[:, feat2, m, nb, n_a] * cred, axis=1)
            cred_cum_input += cred

            if m < M - 1:
                if expert == "EWA":
                    W_output_not_renorm[:, :, m + 1] = EWA(
                        W_output_not_renorm[:, :, m], eta_output, cred, K_output
                    )
                elif expert == "Multilin":
                    W_output_not_renorm[:,:,m+1]=Multilin(W_output_not_renorm[:,:,m],eta_output,list_cred_output[:,:,m],K_output)
                elif expert == "PWA":
                    W_output_not_renorm[:,:,m+1]=PWA(para_PWA,K_output,K_input_new,cred_cum_output,cred_cum_input)
                
                
                if np.sum(W_output_not_renorm[:,:,m+1])==0:
                    W_output[:, feat2, m + 1, nb, n_a] = 1/K_input_new

                else: 
                    W_output[:, feat2, m + 1, nb, n_a] = (
                        W_output_not_renorm[:, :, m + 1]
                        / np.sum(W_output_not_renorm[:, :, m + 1], axis=1)[:, np.newaxis]
                    )
                #print(W_output_not_renorm[:,:,m+1])
                #print(W_output[:, feat2, m + 1, nb, n_a])
        
    print(n_a)
        

np.save(f"{expert}_{learning}_HAN_ablation",answers)
np.save(f"{expert}_{learning}_W_HAN_ablation", W_output)


In [ ]:
#print(probas)
print(m)
print(n_a)
print(nb)
print(W_output_not_renorm[:, :, m])
print(W_output[:, feat2, m-1, nb, n_a])
print(cred_cum_output)
print(cred_cum_input)